In [ ]:
import os
import pickle
import json
import random
import numpy as np
!pip install torch
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
# set seed
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)

In [ ]:
import warnings
warnings.filterwarnings("ignore")

DATA_PATH = "data"

In [ ]:
!pip install torch_geometric --user

In [ ]:
from torch_geometric.data import Data

In [ ]:
def graph_stat(dataset):
    """
    TODO: calculate the statistics of the ENZYMES dataset.
    
    Outputs:
        min_num_nodes: min number of nodes
        max_num_nodes: max number of nodes
        mean_num_nodes: average number of nodes
        min_num_edges: min number of edges
        max_num_edges: max number of edges
        mean_num_edges: average number of edges
    """
    nodes_edges = [(data.num_nodes, data.num_edges) for data in dataset]
    num_nodes, num_edges = list(list(zip(*nodes_edges))[0]), list(list(zip(*nodes_edges))[1])
    min_num_nodes = min(num_nodes)
    max_num_nodes = max(num_nodes)
    mean_num_nodes = np.mean(num_nodes)
    min_num_edges = min(num_edges)
    max_num_edges = max(num_edges)
    mean_num_edges = np.mean(num_edges)
    
    return min_num_nodes, max_num_nodes, mean_num_nodes, \
            min_num_edges, max_num_edges, mean_num_edges
    

In [ ]:
import math
from torch_geometric.utils import to_dense_adj, add_self_loops

class GCNConv(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GCNConv, self).__init__()
        self.theta = nn.Parameter(torch.FloatTensor(in_channels, out_channels))
        # Initialize the parameters.
        stdv = 1. / math.sqrt(out_channels)
        self.theta.data.uniform_(-stdv, stdv)
    
    def forward(self, x, edge_index):
        """
        TODO:
            1. Generate the adjacency matrix with self-loop \hat{A} using edge_index.
            2. Calculate the diagonal degree matrix \hat{D}.
            3. Calculate the output X' with torch.mm using the equation above.
        """

        num_nodes = x.shape[0]
        A = torch.sparse_coo_tensor(edge_index, torch.ones(edge_index.shape[1]), (num_nodes, num_nodes))
        A = A.to_dense()
        A_hat = A + torch.eye(num_nodes)
        
        A_sum = torch.sum(A_hat, dim=1)
        D = torch.pow(A_sum, -0.5)
        D[D == float('inf')] = 0.0
        D_hat_sqrt = torch.diag(D)
        
        first = torch.mm(torch.mm(D_hat_sqrt, A_hat), D_hat_sqrt)
        second = torch.mm(x, self.theta)
        
        ret = torch.mm(first, second)
        
        return ret

In [ ]:
# from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool

class GCN(torch.nn.Module):
    def __init__(self):
        super(GCN, self).__init__()
        """
        TODO:
            1. Define the first convolution layer using `GCNConv()`. Set `out_channels` to 64;
            2. Define the first activation layer using `nn.ReLU()`;
            3. Define the second convolution layer using `GCNConv()`. Set `out_channels` to 64;
            4. Define the second activation layer using `nn.ReLU()`;
            5. Define the third convolution layer using `GCNConv()`. Set `out_channels` to 64;
            6. Define the dropout layer using `nn.Dropout()`;
            7. Define the linear layer using `nn.Linear()`. Set `output_size` to 2.

        Note that for MUTAG dataset, the number of node features is 7, and the number of classes is 2.

        """
        
        self.gcn1 = GCNConv(in_channels=7, out_channels=64)
        self.a1 = nn.ReLU()
        self.gcn2 = GCNConv(in_channels=64, out_channels=64)
        self.a2 = nn.ReLU()
        self.gcn3 = GCNConv(in_channels=64, out_channels=64)
        self.dropout = nn.Dropout(p=0.4)
        self.linear = nn.Linear(in_features=64, out_features=2)

    def forward(self, x, edge_index, batch):
        """
        TODO:
            1. Pass the data through the frst convolution layer;
            2. Pass the data through the activation layer;
            3. Pass the data through the second convolution layer;
            4. Obtain the graph embeddings using the readout layer with `global_mean_pool()`;
            5. Pass the graph embeddgins through the dropout layer;
            6. Pass the graph embeddings through the linear layer.
            
        Arguments:
            x: [num_nodes, 7], node features
            edge_index: [2, num_edges], edges
            batch: [num_nodes], batch assignment vector which maps each node to its 
                   respective graph in the batch

        Outputs:
            probs: probabilities of shape (batch_size, 2)
        """
        x = self.gcn1(x, edge_index)
        x = self.a1(x)
        x = self.gcn2(x, edge_index)
        x = self.a2(x)
        x = self.gcn3(x, edge_index)
        x = global_mean_pool(x, batch)
        x = self.dropout(x)
        x = self.linear(x)
        
        probs = torch.nn.functional.softmax(x, dim=-1)
        
        return probs
        
        
        
GCN()

In [ ]:
gcn = GCN()

# optimizer
optimizer = torch.optim.Adam(gcn.parameters(), lr=0.01)
# loss
criterion = torch.nn.CrossEntropyLoss()

def train(train_loader):
    gcn.train()
    for data in train_loader:  # Iterate in batches over the training dataset.
        """
        TODO: train the model for one epoch.
        
        Note that you can acess the batch data using `data.x`, `data.edge_index`, `data.batch`, `data,y`.
        """
        
        out = gcn(data.x, data.edge_index, data.batch)
        loss = criterion(out, data.y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        
def test(loader):
    gcn.eval()
    correct = 0
    for data in loader:  # Iterate in batches over the training/test dataset.
        out = gcn(data.x, data.edge_index, data.batch)  
        pred = out.argmax(dim=1)  # Use the class with highest probability.
        correct += int((pred == data.y).sum())  # Check against ground-truth labels.
    return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(200):
    train(train_loader)
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch + 1:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')